In [18]:
import numpy as np
import pandas as pd
from pathlib import Path
from lightning import pytorch as pl
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from chemprop import data, featurizers, models, nn, utils
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from lightning.pytorch.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from openpyxl import Workbook
import os
from pathlib import Path
import sys
print('Python:',sys.version)

Python: 3.11.10 (main, Oct  3 2024, 07:29:13) [GCC 11.2.0]


# Full dataset

In [2]:
df = pd.read_excel("scCO2_all_data(curated).xlsx")

In [3]:
df_rem = df[~df['Name'].isin(['Water', 'Iodine'])] #remove water and iodine from the data set

In [4]:
smiles_column = ['SMILES_Canonical']
target_columns = ['Lg(y)']
descriptor_columns = ['T (K)', 'P (bar)', 'Melting Point', 'dHvap (kJ/mol)', 'g (gcm3)', 'dG', 'MolWt', 'MolLogP', 'TPSA', 'NR', 'Bj', 'HA', 'HD', 'RC', 'SP', 'NA', 'NAr', 'Hat']

In [5]:
smis = df_rem.loc[:, smiles_column].values.flatten().astype(str)
ys = df_rem.loc[:, target_columns].values
extra_datapoint_descriptors = df_rem[descriptor_columns].values

In [6]:
all_data = [
    data.MoleculeDatapoint(Chem.MolFromSmiles(smi), y, x_d=X_d)
    for smi, y, X_d in zip(smis, ys, extra_datapoint_descriptors)
]

In [7]:
mols = [d.mol for d in all_data]

In [8]:
#Strict split SMILES. D-MPNN with thermodynamic properties

In [9]:
class ScaffoldGroupKFold:
    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        unique_groups = np.unique(groups)
        if self.shuffle:
            np.random.seed(self.random_state)
            np.random.shuffle(unique_groups)
        
        fold_size = len(unique_groups) // self.n_splits
        
        for i in range(self.n_splits):
            if i == self.n_splits - 1:
                test_groups = unique_groups[i * fold_size:]
            else:
                test_groups = unique_groups[i * fold_size: (i + 1) * fold_size]
            
            test_indices = np.where(np.isin(groups, test_groups))[0]
            train_indices = np.where(~np.isin(groups, test_groups))[0]
            
            yield train_indices, test_indices

In [10]:
k_splits = ScaffoldGroupKFold(n_splits=5, shuffle=True, random_state=4321)
output_path = Path.cwd() / "hup" / "crossval_results_scCO2.xlsx"

In [11]:
def calculate_aard(true_values, predicted_values):
    true_values = np.array(true_values)  
    predicted_values = np.array(predicted_values)
    return 100 * np.mean(np.abs((true_values - predicted_values) / true_values))

def calculate_r2(true_values, predicted_values):
    true_values = np.array(true_values)  
    predicted_values = np.array(predicted_values)
    return r2_score(true_values, predicted_values)

def calculate_rmse(true_values, predicted_values):
    return np.sqrt(mean_squared_error(true_values, predicted_values))

def calculate_mae(true_values, predicted_values):
    return mean_absolute_error(true_values, predicted_values)

In [12]:
fold_results = {"RMSE": [], "MAE": [], "R²": [], "AARD": []}
all_test_smiles = []
all_test_targets = []
all_test_predictions = []

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    pd.DataFrame({"Placeholder": []}).to_excel(writer, sheet_name="Init", index=False)

    for fold_idx, (train_indices, test_indices) in enumerate(k_splits.split(mols, groups=df_rem['SMILES_Canonical'])):
        print(f"Starting fold {fold_idx + 1}/5")

        train_data, _, test_data = data.split_data_by_indices(all_data, [train_indices], None, [test_indices])
        train_data, val_data = train_test_split(train_data[0], test_size=0.05, random_state=fold_idx)

        featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
        train_dset = data.MoleculeDataset(train_data, featurizer)
        scaler = train_dset.normalize_targets()

        val_dset = data.MoleculeDataset(val_data, featurizer)
        val_dset.normalize_targets(scaler)

        test_dset = data.MoleculeDataset(test_data[0], featurizer)
        
        targets_scaler = train_dset.normalize_targets()
        extra_datapoint_descriptors_scaler = train_dset.normalize_inputs("X_d")
        val_dset.normalize_targets(targets_scaler)
        val_dset.normalize_inputs("X_d", extra_datapoint_descriptors_scaler)
        
        train_dset.cache = True
        val_dset.cache = True

        
        train_loader = data.build_dataloader(train_dset, shuffle=True)
        val_loader = data.build_dataloader(val_dset, shuffle=False)
        test_loader = data.build_dataloader(test_dset, shuffle=False)

        # optimized by Ray Tune
        ffn_hidden_dim = 2400
        message_hidden_dim = 300
        depth = 2
        ffn_num_layers = 1
        batch_norm = True
        metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
        descriptor_columns = extra_datapoint_descriptors.shape[1]

        mp = nn.BondMessagePassing(d_h=message_hidden_dim)
        agg = nn.MeanAggregation()
        output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
        ffn = nn.RegressionFFN(
            output_transform=output_transform,
            input_dim=message_hidden_dim + descriptor_columns,
            hidden_dim=ffn_hidden_dim,
            n_layers=ffn_num_layers
        )
        X_d_transform = nn.ScaleTransform.from_standard_scaler(extra_datapoint_descriptors_scaler)

        model = models.MPNN(
            mp, agg, ffn, batch_norm, metric_list, X_d_transform=X_d_transform
        )
        
        # Checkpointing
        checkpoint_dir = Path(f"/home/lab101/temp/mdm/bot/chek/fold_{fold_idx}")
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        checkpointing = ModelCheckpoint(
            dirpath=checkpoint_dir,
            filename="best-{epoch}-{val_loss:.2f}",
            monitor="val_loss",
            mode="min",
            save_weights_only=True
        )

        trainer = pl.Trainer(accelerator="auto", devices=1, max_epochs=15, callbacks=[checkpointing])
        trainer.fit(model, train_loader, val_loader)

        predictions = trainer.predict(model, test_loader)
        test_smiles = [Chem.MolToSmiles(d.mol) for d in test_data[0]]
        test_targets = [d.y[0] for d in test_data[0]]
        fold_predictions = np.concatenate(predictions).flatten().tolist()

        all_test_smiles.extend(test_smiles)
        all_test_targets.extend(test_targets)
        all_test_predictions.extend(fold_predictions)

        rmse = calculate_rmse(test_targets, fold_predictions)
        mae = calculate_mae(test_targets, fold_predictions)
        r2 = r2_score(test_targets, fold_predictions)
        aard = calculate_aard(test_targets, fold_predictions)

        fold_results["RMSE"].append(rmse)
        fold_results["MAE"].append(mae)
        fold_results["R²"].append(r2)
        fold_results["AARD"].append(aard)

        df_fold = pd.DataFrame({"SMILES": test_smiles, "True_Target": test_targets, "Predicted_Target": fold_predictions})
        df_fold.to_excel(writer, sheet_name=f"Fold_{fold_idx + 1}", index=False)

        print(f"Fold {fold_idx + 1}:")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R²: {r2}")
        print(f"  AARD, %: {aard}")

mean_rmse = np.mean(fold_results["RMSE"])
std_rmse = np.std(fold_results["RMSE"])
mean_mae = np.mean(fold_results["MAE"])
std_mae = np.std(fold_results["MAE"])
mean_r2 = np.mean(fold_results["R²"])
std_r2 = np.std(fold_results["R²"])
mean_aard = np.mean(fold_results["AARD"])
std_aard = np.std(fold_results["AARD"])

print("\nFinal cross-validation results:")
print(f"  RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"  MAE: {mean_mae:.4f} ± {std_mae:.4f}")
print(f"  R²: {mean_r2:.4f} ± {std_r2:.4f}")
print(f"  AARD, %: {mean_aard:.4f} ± {std_aard:.4f}")

df_results = pd.DataFrame({
    "SMILES": all_test_smiles,
    "True_Target": all_test_targets,
    "Predicted_Target": all_test_predictions
})
with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
    df_results.to_excel(writer, sheet_name="All_Folds", index=False)

Starting fold 1/5


/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/chemprop/data/dataloader.py:52: UserWarning: Dropping last batch of size 1 to avoid issues with batch normalization (dataset size = 1217, batch_size = 64)
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/lab101/temp/mdm/bot/chek/fold_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loading `train_dataloader` to estimate numb

Sanity Checking DataLoader 0:   0%|                       | 0/2 [00:00<?, ?it/s]

/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Epoch 0: 100%|█| 362/362 [00:30<00:00, 12.02it/s, v_num=515, train_loss_step=0.4
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 19/19 [00:01<00:00, 12.55it/s]
Epoch 1: 100%|█| 362/362 [00:33<00:00, 10.83it/s, v_num=515, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 19/19 [00:01<00:00, 13.50it/s]
Epoch 2: 100%|█| 362/362 [00:29<00:00, 12.42it/s, v_num=515, train_loss_step=0.2
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 19/19 [00:01<00:00, 12.70it/s]
Epoch 3: 100%|█| 362/362 [00:27<00:00, 12.97it/s, v_num=515, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 19/19 [00:01<00:00, 13.66it/s]
Epoch 4: 100%|█| 362/362 [00

...

Validation DataLoader 0: 100%|██████████████████| 20/20 [00:01<00:00, 10.97it/s]
Epoch 9: 100%|█| 379/379 [00:31<00:00, 11.92it/s, v_num=519, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 20/20 [00:01<00:00, 12.93it/s]
Epoch 10: 100%|█| 379/379 [00:31<00:00, 11.91it/s, v_num=519, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 20/20 [00:01<00:00, 13.23it/s]
Epoch 11: 100%|█| 379/379 [00:31<00:00, 11.99it/s, v_num=519, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 20/20 [00:01<00:00, 13.98it/s]
Epoch 12: 100%|█| 379/379 [00:31<00:00, 11.85it/s, v_num=519, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|█| 379/379 [00:33<00:00, 11.36it/s, v_num=519, train_loss_step=0.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████████████| 80/80 [00:22<00:00,  3.51it/s]
Fold 5:
  RMSE: 0.8665760187567846
  MAE: 0.6573915761676988
  R²: 0.5904694231228433
  AARD, %: 19.76875547047459

Final cross-validation results:
  RMSE: 0.7650 ± 0.0528
  MAE: 0.5582 ± 0.0525
  R²: 0.7039 ± 0.0586
  AARD, %: 17.0330 ± 1.6949


In [13]:
#Scaffold split. D-MPNN with thermodynamic properties

In [15]:
def get_scaffold(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return MurckoScaffold.MurckoScaffoldSmiles(mol=mol)

df_rem['scaffold'] = df_rem['SMILES_Canonical'].apply(get_scaffold)

/tmp/ipykernel_1821628/3960520845.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rem['scaffold'] = df_rem['SMILES_Canonical'].apply(get_scaffold)


In [16]:
df_rem['scaffold_group'] = df_rem['scaffold'].astype('category').cat.codes

/tmp/ipykernel_1821628/1701185365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rem['scaffold_group'] = df_rem['scaffold'].astype('category').cat.codes


In [17]:
fold_results = {"RMSE": [], "MAE": [], "R²": [], "AARD": []}
all_test_smiles = []
all_test_targets = []
all_test_predictions = []

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    pd.DataFrame({"Placeholder": []}).to_excel(writer, sheet_name="Init", index=False)

    for fold_idx, (train_indices, test_indices) in enumerate(k_splits.split(mols, groups=df_rem['scaffold_group'])):
        print(f"Starting fold {fold_idx + 1}/5")

        train_data, _, test_data = data.split_data_by_indices(all_data, [train_indices], None, [test_indices])
        train_data, val_data = train_test_split(train_data[0], test_size=0.05, random_state=fold_idx)

        featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
        train_dset = data.MoleculeDataset(train_data, featurizer)
        scaler = train_dset.normalize_targets()

        val_dset = data.MoleculeDataset(val_data, featurizer)
        val_dset.normalize_targets(scaler)

        test_dset = data.MoleculeDataset(test_data[0], featurizer)
        
        targets_scaler = train_dset.normalize_targets()
        extra_datapoint_descriptors_scaler = train_dset.normalize_inputs("X_d")
        val_dset.normalize_targets(targets_scaler)
        val_dset.normalize_inputs("X_d", extra_datapoint_descriptors_scaler)
        
        train_dset.cache = True
        val_dset.cache = True

        
        train_loader = data.build_dataloader(train_dset, shuffle=True)
        val_loader = data.build_dataloader(val_dset, shuffle=False)
        test_loader = data.build_dataloader(test_dset, shuffle=False)

        # optimized by Ray Tune
        ffn_hidden_dim = 2400
        message_hidden_dim = 300
        depth = 2
        ffn_num_layers = 1
        batch_norm = True
        metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
        descriptor_columns = extra_datapoint_descriptors.shape[1]

        mp = nn.BondMessagePassing(d_h=message_hidden_dim)
        agg = nn.MeanAggregation()
        output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
        ffn = nn.RegressionFFN(
            output_transform=output_transform,
            input_dim=message_hidden_dim + descriptor_columns,
            hidden_dim=ffn_hidden_dim,
            n_layers=ffn_num_layers
        )
        X_d_transform = nn.ScaleTransform.from_standard_scaler(extra_datapoint_descriptors_scaler)

        model = models.MPNN(
            mp, agg, ffn, batch_norm, metric_list, X_d_transform=X_d_transform
        )
        
        # Checkpointing
        checkpoint_dir = Path(f"/home/lab101/temp/mdm/bot/chek/fold_{fold_idx}")
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        checkpointing = ModelCheckpoint(
            dirpath=checkpoint_dir,
            filename="best-{epoch}-{val_loss:.2f}",
            monitor="val_loss",
            mode="min",
            save_weights_only=True
        )

        trainer = pl.Trainer(accelerator="auto", devices=1, max_epochs=15, callbacks=[checkpointing])
        trainer.fit(model, train_loader, val_loader)

        predictions = trainer.predict(model, test_loader)
        test_smiles = [Chem.MolToSmiles(d.mol) for d in test_data[0]]
        test_targets = [d.y[0] for d in test_data[0]]
        fold_predictions = np.concatenate(predictions).flatten().tolist()

        all_test_smiles.extend(test_smiles)
        all_test_targets.extend(test_targets)
        all_test_predictions.extend(fold_predictions)

        rmse = calculate_rmse(test_targets, fold_predictions)
        mae = calculate_mae(test_targets, fold_predictions)
        r2 = r2_score(test_targets, fold_predictions)
        aard = calculate_aard(test_targets, fold_predictions)

        fold_results["RMSE"].append(rmse)
        fold_results["MAE"].append(mae)
        fold_results["R²"].append(r2)
        fold_results["AARD"].append(aard)

        df_fold = pd.DataFrame({"SMILES": test_smiles, "True_Target": test_targets, "Predicted_Target": fold_predictions})
        df_fold.to_excel(writer, sheet_name=f"Fold_{fold_idx + 1}", index=False)

        print(f"Fold {fold_idx + 1}:")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R²: {r2}")
        print(f"  AARD, %: {aard}")

mean_rmse = np.mean(fold_results["RMSE"])
std_rmse = np.std(fold_results["RMSE"])
mean_mae = np.mean(fold_results["MAE"])
std_mae = np.std(fold_results["MAE"])
mean_r2 = np.mean(fold_results["R²"])
std_r2 = np.std(fold_results["R²"])
mean_aard = np.mean(fold_results["AARD"])
std_aard = np.std(fold_results["AARD"])

print("\nFinal cross-validation results:")
print(f"  RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"  MAE: {mean_mae:.4f} ± {std_mae:.4f}")
print(f"  R²: {mean_r2:.4f} ± {std_r2:.4f}")
print(f"  AARD, %: {mean_aard:.4f} ± {std_aard:.4f}")

df_results = pd.DataFrame({
    "SMILES": all_test_smiles,
    "True_Target": all_test_targets,
    "Predicted_Target": all_test_predictions
})
with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
    df_results.to_excel(writer, sheet_name="All_Folds", index=False)

Starting fold 1/5


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/lab101/temp/mdm/bot/chek/fold_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loading `train_dataloader` to estimate number of stepping batches.
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn    

Sanity Checking DataLoader 0:   0%|                       | 0/2 [00:00<?, ?it/s]

/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Epoch 0: 100%|█| 400/400 [00:33<00:00, 12.11it/s, v_num=520, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 22/22 [00:00<00:00, 27.35it/s]
Epoch 1: 100%|█| 400/400 [00:26<00:00, 15.16it/s, v_num=520, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 22/22 [00:01<00:00, 12.64it/s]
Epoch 2: 100%|█| 400/400 [00:32<00:00, 12.28it/s, v_num=520, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 22/22 [00:01<00:00, 12.51it/s]
Epoch 3: 100%|█| 400/400 [00:32<00:00, 12.38it/s, v_num=520, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 22/22 [00:01<00:00, 12.02it/s]
Epoch 4: 100%|█| 400/400 [00

...

Validation DataLoader 0: 100%|██████████████████| 18/18 [00:01<00:00, 11.34it/s]
Epoch 9: 100%|█| 334/334 [00:28<00:00, 11.92it/s, v_num=524, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 18/18 [00:01<00:00, 11.96it/s]
Epoch 10: 100%|█| 334/334 [00:27<00:00, 12.25it/s, v_num=524, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 18/18 [00:01<00:00, 14.68it/s]
Epoch 11: 100%|█| 334/334 [00:27<00:00, 12.34it/s, v_num=524, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 18/18 [00:01<00:00, 12.57it/s]
Epoch 12: 100%|█| 334/334 [00:26<00:00, 12.67it/s, v_num=524, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|█| 334/334 [00:28<00:00, 11.89it/s, v_num=524, train_loss_step=0.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.



Predicting DataLoader 0: 100%|████████████████| 128/128 [00:29<00:00,  4.33it/s]
Fold 5:
  RMSE: 0.9141963702446654
  MAE: 0.679938863693288
  R²: 0.5328537399088338
  AARD, %: 23.03348859847843

Final cross-validation results:
  RMSE: 0.8742 ± 0.0772
  MAE: 0.6430 ± 0.0664
  R²: 0.5606 ± 0.0343
  AARD, %: 18.2421 ± 4.7719


# Drug-like dataset

In [19]:
df = pd.read_excel("Drug-like_data.xlsx")

In [20]:
smiles_column = ['SMILES_Canonical']
target_columns = ['Lg(y)']
descriptor_columns = ['T (K)', 'P (bar)', 'Melting Point', 'dHvap (kJ/mol)', 'g (gcm3)', 'dG', 'MolWt', 'MolLogP', 'TPSA', 'NR', 'Bj', 'HA', 'HD', 'RC', 'SP', 'NA', 'NAr', 'Hat']

In [21]:
smis = df.loc[:, smiles_column].values.flatten().astype(str)
ys = df.loc[:, target_columns].values
extra_datapoint_descriptors = df[descriptor_columns].values

In [22]:
all_data = [
    data.MoleculeDatapoint(Chem.MolFromSmiles(smi), y, x_d=X_d)
    for smi, y, X_d in zip(smis, ys, extra_datapoint_descriptors)
]

In [23]:
mols = [d.mol for d in all_data]

In [24]:
#Strict split SMILES. D-MPNN with thermodynamic properties

In [25]:
class ScaffoldGroupKFold:
    def __init__(self, n_splits=5, shuffle=False, random_state=None):
        self.n_splits = n_splits
        self.shuffle = shuffle
        self.random_state = random_state

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        unique_groups = np.unique(groups)
        if self.shuffle:
            np.random.seed(self.random_state)
            np.random.shuffle(unique_groups)
        
        fold_size = len(unique_groups) // self.n_splits
        
        for i in range(self.n_splits):
            if i == self.n_splits - 1:
                test_groups = unique_groups[i * fold_size:]
            else:
                test_groups = unique_groups[i * fold_size: (i + 1) * fold_size]
            
            test_indices = np.where(np.isin(groups, test_groups))[0]
            train_indices = np.where(~np.isin(groups, test_groups))[0]
            
            yield train_indices, test_indices

In [26]:
k_splits = ScaffoldGroupKFold(n_splits=5, shuffle=True, random_state=4321)
output_path = Path.cwd() / "hup" / "crossval_results_scCO2.xlsx"

In [27]:
def calculate_aard(true_values, predicted_values):
    true_values = np.array(true_values)  
    predicted_values = np.array(predicted_values)
    return 100 * np.mean(np.abs((true_values - predicted_values) / true_values))

def calculate_r2(true_values, predicted_values):
    true_values = np.array(true_values)  
    predicted_values = np.array(predicted_values)
    return r2_score(true_values, predicted_values)

def calculate_rmse(true_values, predicted_values):
    return np.sqrt(mean_squared_error(true_values, predicted_values))

def calculate_mae(true_values, predicted_values):
    return mean_absolute_error(true_values, predicted_values)

In [28]:
fold_results = {"RMSE": [], "MAE": [], "R²": [], "AARD": []}
all_test_smiles = []
all_test_targets = []
all_test_predictions = []

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    pd.DataFrame({"Placeholder": []}).to_excel(writer, sheet_name="Init", index=False)

    for fold_idx, (train_indices, test_indices) in enumerate(k_splits.split(mols, groups=df['SMILES_Canonical'])):
        print(f"Starting fold {fold_idx + 1}/5")

        train_data, _, test_data = data.split_data_by_indices(all_data, [train_indices], None, [test_indices])
        train_data, val_data = train_test_split(train_data[0], test_size=0.05, random_state=fold_idx)

        featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
        train_dset = data.MoleculeDataset(train_data, featurizer)
        scaler = train_dset.normalize_targets()

        val_dset = data.MoleculeDataset(val_data, featurizer)
        val_dset.normalize_targets(scaler)

        test_dset = data.MoleculeDataset(test_data[0], featurizer)
        
        targets_scaler = train_dset.normalize_targets()
        extra_datapoint_descriptors_scaler = train_dset.normalize_inputs("X_d")
        val_dset.normalize_targets(targets_scaler)
        val_dset.normalize_inputs("X_d", extra_datapoint_descriptors_scaler)
        
        train_dset.cache = True
        val_dset.cache = True

        
        train_loader = data.build_dataloader(train_dset, shuffle=True)
        val_loader = data.build_dataloader(val_dset, shuffle=False)
        test_loader = data.build_dataloader(test_dset, shuffle=False)

        # optimized by Ray Tune
        ffn_hidden_dim = 2400
        message_hidden_dim = 300
        depth = 2
        ffn_num_layers = 1
        batch_norm = True
        metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
        descriptor_columns = extra_datapoint_descriptors.shape[1]

        mp = nn.BondMessagePassing(d_h=message_hidden_dim)
        agg = nn.MeanAggregation()
        output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
        ffn = nn.RegressionFFN(
            output_transform=output_transform,
            input_dim=message_hidden_dim + descriptor_columns,
            hidden_dim=ffn_hidden_dim,
            n_layers=ffn_num_layers
        )
        X_d_transform = nn.ScaleTransform.from_standard_scaler(extra_datapoint_descriptors_scaler)

        model = models.MPNN(
            mp, agg, ffn, batch_norm, metric_list, X_d_transform=X_d_transform
        )
        
        # Checkpointing
        checkpoint_dir = Path(f"/home/lab101/temp/mdm/bot/chek/fold_{fold_idx}")
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        checkpointing = ModelCheckpoint(
            dirpath=checkpoint_dir,
            filename="best-{epoch}-{val_loss:.2f}",
            monitor="val_loss",
            mode="min",
            save_weights_only=True
        )

        trainer = pl.Trainer(accelerator="auto", devices=1, max_epochs=15, callbacks=[checkpointing])
        trainer.fit(model, train_loader, val_loader)

        predictions = trainer.predict(model, test_loader)
        test_smiles = [Chem.MolToSmiles(d.mol) for d in test_data[0]]
        test_targets = [d.y[0] for d in test_data[0]]
        fold_predictions = np.concatenate(predictions).flatten().tolist()

        all_test_smiles.extend(test_smiles)
        all_test_targets.extend(test_targets)
        all_test_predictions.extend(fold_predictions)

        rmse = calculate_rmse(test_targets, fold_predictions)
        mae = calculate_mae(test_targets, fold_predictions)
        r2 = r2_score(test_targets, fold_predictions)
        aard = calculate_aard(test_targets, fold_predictions)

        fold_results["RMSE"].append(rmse)
        fold_results["MAE"].append(mae)
        fold_results["R²"].append(r2)
        fold_results["AARD"].append(aard)

        df_fold = pd.DataFrame({"SMILES": test_smiles, "True_Target": test_targets, "Predicted_Target": fold_predictions})
        df_fold.to_excel(writer, sheet_name=f"Fold_{fold_idx + 1}", index=False)

        print(f"Fold {fold_idx + 1}:")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R²: {r2}")
        print(f"  AARD, %: {aard}")

mean_rmse = np.mean(fold_results["RMSE"])
std_rmse = np.std(fold_results["RMSE"])
mean_mae = np.mean(fold_results["MAE"])
std_mae = np.std(fold_results["MAE"])
mean_r2 = np.mean(fold_results["R²"])
std_r2 = np.std(fold_results["R²"])
mean_aard = np.mean(fold_results["AARD"])
std_aard = np.std(fold_results["AARD"])

print("\nFinal cross-validation results:")
print(f"  RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"  MAE: {mean_mae:.4f} ± {std_mae:.4f}")
print(f"  R²: {mean_r2:.4f} ± {std_r2:.4f}")
print(f"  AARD, %: {mean_aard:.4f} ± {std_aard:.4f}")

df_results = pd.DataFrame({
    "SMILES": all_test_smiles,
    "True_Target": all_test_targets,
    "Predicted_Target": all_test_predictions
})
with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
    df_results.to_excel(writer, sheet_name="All_Folds", index=False)

Starting fold 1/5


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/lab101/temp/mdm/bot/chek/fold_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loading `train_dataloader` to estimate number of stepping batches.
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn    

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Epoch 0: 100%|█| 238/238 [00:14<00:00, 15.97it/s, v_num=525, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 19.32it/s]
Epoch 1: 100%|█| 238/238 [00:15<00:00, 15.79it/s, v_num=525, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 17.13it/s]
Epoch 2: 100%|█| 238/238 [00:15<00:00, 15.69it/s, v_num=525, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 16.74it/s]
Epoch 3: 100%|█| 238/238 [00:15<00:00, 15.31it/s, v_num=525, train_loss_step=0.3
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 17.47it/s]
Epoch 4: 100%|█| 238/238 [00

...

Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 14.26it/s]
Epoch 12: 100%|█| 242/242 [00:16<00:00, 15.12it/s, v_num=529, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 14.32it/s]
Epoch 13: 100%|█| 242/242 [00:16<00:00, 14.34it/s, v_num=529, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 14.28it/s]
Epoch 14: 100%|█| 242/242 [00:16<00:00, 14.80it/s, v_num=529, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 13/13 [00:00<00:00, 14.46it/s]
Epoch 14: 100%|█| 242/242 [00:17<00:00, 14.00it/s, v_num=529, train_loss_step=0.

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|█| 242/242 [00:17<00:00, 13.96it/s, v_num=529, train_loss_step=0.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████████████| 56/56 [00:12<00:00,  4.58it/s]
Fold 5:
  RMSE: 0.6685365068959032
  MAE: 0.4940107034571468
  R²: 0.7623929450218354
  AARD, %: 15.413898174092754

Final cross-validation results:
  RMSE: 0.7234 ± 0.0393
  MAE: 0.5429 ± 0.0336
  R²: 0.7324 ± 0.0365
  AARD, %: 17.1160 ± 2.5055


In [29]:
#Scaffold split. D-MPNN with thermodynamic properties

In [30]:
def get_scaffold(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return MurckoScaffold.MurckoScaffoldSmiles(mol=mol)

df['scaffold'] = df['SMILES_Canonical'].apply(get_scaffold)

In [31]:
df['scaffold_group'] = df['scaffold'].astype('category').cat.codes

In [32]:
fold_results = {"RMSE": [], "MAE": [], "R²": [], "AARD": []}
all_test_smiles = []
all_test_targets = []
all_test_predictions = []

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    pd.DataFrame({"Placeholder": []}).to_excel(writer, sheet_name="Init", index=False)

    for fold_idx, (train_indices, test_indices) in enumerate(k_splits.split(mols, groups=df['scaffold_group'])):
        print(f"Starting fold {fold_idx + 1}/5")

        train_data, _, test_data = data.split_data_by_indices(all_data, [train_indices], None, [test_indices])
        train_data, val_data = train_test_split(train_data[0], test_size=0.05, random_state=fold_idx)

        featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()
        train_dset = data.MoleculeDataset(train_data, featurizer)
        scaler = train_dset.normalize_targets()

        val_dset = data.MoleculeDataset(val_data, featurizer)
        val_dset.normalize_targets(scaler)

        test_dset = data.MoleculeDataset(test_data[0], featurizer)
        
        targets_scaler = train_dset.normalize_targets()
        extra_datapoint_descriptors_scaler = train_dset.normalize_inputs("X_d")
        val_dset.normalize_targets(targets_scaler)
        val_dset.normalize_inputs("X_d", extra_datapoint_descriptors_scaler)
        
        train_dset.cache = True
        val_dset.cache = True

        
        train_loader = data.build_dataloader(train_dset, shuffle=True)
        val_loader = data.build_dataloader(val_dset, shuffle=False)
        test_loader = data.build_dataloader(test_dset, shuffle=False)

        # optimized by Ray Tune
        ffn_hidden_dim = 2400
        message_hidden_dim = 300
        depth = 2
        ffn_num_layers = 1
        batch_norm = True
        metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
        descriptor_columns = extra_datapoint_descriptors.shape[1]

        mp = nn.BondMessagePassing(d_h=message_hidden_dim)
        agg = nn.MeanAggregation()
        output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
        ffn = nn.RegressionFFN(
            output_transform=output_transform,
            input_dim=message_hidden_dim + descriptor_columns,
            hidden_dim=ffn_hidden_dim,
            n_layers=ffn_num_layers
        )
        X_d_transform = nn.ScaleTransform.from_standard_scaler(extra_datapoint_descriptors_scaler)

        model = models.MPNN(
            mp, agg, ffn, batch_norm, metric_list, X_d_transform=X_d_transform
        )
        
        # Checkpointing
        checkpoint_dir = Path(f"/home/lab101/temp/mdm/bot/chek/fold_{fold_idx}")
        checkpoint_dir.mkdir(parents=True, exist_ok=True)

        checkpointing = ModelCheckpoint(
            dirpath=checkpoint_dir,
            filename="best-{epoch}-{val_loss:.2f}",
            monitor="val_loss",
            mode="min",
            save_weights_only=True
        )

        trainer = pl.Trainer(accelerator="auto", devices=1, max_epochs=15, callbacks=[checkpointing])
        trainer.fit(model, train_loader, val_loader)

        predictions = trainer.predict(model, test_loader)
        test_smiles = [Chem.MolToSmiles(d.mol) for d in test_data[0]]
        test_targets = [d.y[0] for d in test_data[0]]
        fold_predictions = np.concatenate(predictions).flatten().tolist()

        all_test_smiles.extend(test_smiles)
        all_test_targets.extend(test_targets)
        all_test_predictions.extend(fold_predictions)

        rmse = calculate_rmse(test_targets, fold_predictions)
        mae = calculate_mae(test_targets, fold_predictions)
        r2 = r2_score(test_targets, fold_predictions)
        aard = calculate_aard(test_targets, fold_predictions)

        fold_results["RMSE"].append(rmse)
        fold_results["MAE"].append(mae)
        fold_results["R²"].append(r2)
        fold_results["AARD"].append(aard)

        df_fold = pd.DataFrame({"SMILES": test_smiles, "True_Target": test_targets, "Predicted_Target": fold_predictions})
        df_fold.to_excel(writer, sheet_name=f"Fold_{fold_idx + 1}", index=False)

        print(f"Fold {fold_idx + 1}:")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R²: {r2}")
        print(f"  AARD, %: {aard}")

mean_rmse = np.mean(fold_results["RMSE"])
std_rmse = np.std(fold_results["RMSE"])
mean_mae = np.mean(fold_results["MAE"])
std_mae = np.std(fold_results["MAE"])
mean_r2 = np.mean(fold_results["R²"])
std_r2 = np.std(fold_results["R²"])
mean_aard = np.mean(fold_results["AARD"])
std_aard = np.std(fold_results["AARD"])

print("\nFinal cross-validation results:")
print(f"  RMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"  MAE: {mean_mae:.4f} ± {std_mae:.4f}")
print(f"  R²: {mean_r2:.4f} ± {std_r2:.4f}")
print(f"  AARD, %: {mean_aard:.4f} ± {std_aard:.4f}")

df_results = pd.DataFrame({
    "SMILES": all_test_smiles,
    "True_Target": all_test_targets,
    "Predicted_Target": all_test_predictions
})
with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
    df_results.to_excel(writer, sheet_name="All_Folds", index=False)

Starting fold 1/5


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/lab101/temp/mdm/bot/chek/fold_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loading `train_dataloader` to estimate number of stepping batches.
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.

  | Name            | Type               | Params | Mode 
---------------------------------------------------------------
0 | message_passing | BondMessagePassing | 227 K  | train
1 | agg             | MeanAggregation    | 0      | train
2 | bn    

Sanity Checking DataLoader 0:   0%|                       | 0/2 [00:00<?, ?it/s]

/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Epoch 0: 100%|█| 249/249 [00:15<00:00, 16.31it/s, v_num=530, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 16.31it/s]
Epoch 1: 100%|█| 249/249 [00:15<00:00, 16.17it/s, v_num=530, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 16.16it/s]
Epoch 2: 100%|█| 249/249 [00:16<00:00, 15.18it/s, v_num=530, train_loss_step=0.1
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 15.79it/s]
Epoch 3: 100%|█| 249/249 [00:15<00:00, 15.94it/s, v_num=530, train_loss_step=0.0
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 16.36it/s]
Epoch 4: 100%|█| 249/249 [00

...

Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 16.48it/s]
Epoch 11: 100%|█| 260/260 [00:15<00:00, 16.26it/s, v_num=534, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 17.66it/s]
Epoch 12: 100%|█| 260/260 [00:16<00:00, 15.92it/s, v_num=534, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 16.44it/s]
Epoch 13: 100%|█| 260/260 [00:16<00:00, 15.72it/s, v_num=534, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████████████| 14/14 [00:00<00:00, 15.38it/s]
Epoch 14: 100%|█| 260/260 [00:15<00:00, 17.04it/s, v_num=534, train_loss_step=0.
Validation: |                                             | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|█| 260/260 [00:16<00:00, 16.00it/s, v_num=534, train_loss_step=0.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/lab101/anaconda3/envs/chemp/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=103` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████████████| 37/37 [00:09<00:00,  3.97it/s]
Fold 5:
  RMSE: 0.7895075064775768
  MAE: 0.5975756309140653
  R²: 0.664042825829974
  AARD, %: 15.280018559141432

Final cross-validation results:
  RMSE: 0.8032 ± 0.0924
  MAE: 0.6170 ± 0.0899
  R²: 0.6556 ± 0.0417
  AARD, %: 19.9302 ± 4.9829
